<a href="https://colab.research.google.com/github/Sricharanredd/FMML-226Q1A4408/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1.Yes, averaging the validation accuracy across multiple splits of your dataset can indeed provide more consistent and reliable results compared to relying on a single validation split. This practice is commonly referred to as "cross-validation."

Cross-validation involves splitting your dataset into multiple subsets, typically referred to as "folds," and then training and evaluating your model multiple times, each time using a different fold as the validation set and the remaining folds as the training set. The validation accuracy is recorded for each iteration.

Here are a few reasons why cross-validation can provide more consistent results:

Reduced Variance: By using multiple validation sets, you reduce the impact of random chance in the split. If you happen to have an unusual split in a single validation set that doesn't represent the overall data well, it can lead to misleading accuracy results. Averaging over multiple splits helps mitigate this issue.

Better Estimation of Model Performance: Cross-validation provides a more robust estimate of your model's generalization performance. It gives you a sense of how well your model is likely to perform on unseen data, as it tests the model's performance on different subsets of the data.

Reduced Bias: If your dataset has any inherent ordering or clustering of data points, a single split might inadvertently introduce bias into the validation set. Cross-validation helps ensure that different parts of the dataset are used for validation, reducing the risk of bias.

Common cross-validation techniques include k-fold cross-validation and stratified k-fold cross-validation, where k represents the number of folds or subsets. The results (e.g., validation accuracies) obtained from each fold can be averaged to provide a more stable estimate of model performance.

However, it's important to note that cross-validation can be computationally more expensive because you need to train and evaluate the model multiple times. The choice of the number of folds (k) depends on the size of your dataset and the trade-off between computational cost and the reliability of the results. In practice, 5-fold or 10-fold cross-validation is often used as a good compromise.

In summary, averaging validation accuracy across multiple splits using cross-validation is a valuable practice for obtaining more consistent and reliable performance estimates for your machine learning models. It helps in understanding how well your model generalizes to unseen data and reduces the impact of random variations in data splitting.

2.Cross-validation, including methods like k-fold cross-validation, provides a more accurate estimate of how your model is likely to perform on unseen data compared to a single train-validation split. However, it's essential to understand that the estimate provided by cross-validation is still not the same as the actual test accuracy.

Here's a breakdown of why cross-validation gives a more accurate estimate but is not the same as test accuracy:

Multiple Validation Sets: Cross-validation uses multiple validation sets, which reduces the risk of a single, biased split impacting your evaluation. This helps provide a more reliable estimate of your model's generalization performance.

Model Variability Assessment: By training and evaluating your model multiple times on different subsets of the data, cross-validation gives you a sense of how your model's performance can vary with different data splits. This is valuable for understanding the stability and robustness of your model.

More Data Used for Validation: With k-fold cross-validation, each data point is used for validation exactly once. This means you are using a larger portion of your dataset for validation compared to a single train-validation split, leading to a more comprehensive assessment of your model's performance.

However, it's crucial to keep in mind that cross-validation is still an estimate and not a guarantee of your model's performance on entirely unseen data (i.e., the test data). There could be differences between your cross-validation results and the actual test accuracy for several reasons:

Data Distribution: Cross-validation uses subsets of your dataset, but it may not fully capture the distribution of the test data, especially if the test data is significantly different from the training data.

Data Leakage: In some cases, unintentional data leakage can occur during cross-validation if preprocessing steps or feature engineering are not appropriately applied to each fold. This can lead to overoptimistic estimates of model performance.

Small Datasets: With very small datasets, the differences between cross-validation results and actual test results can be more significant because the subsets used for validation are smaller, and the randomness of the data split plays a larger role.

In practice, cross-validation is a valuable tool for model evaluation and selection, and it provides a more accurate estimate of generalization performance than a single train-validation split. However, it's essential to validate your model on a separate, unseen test dataset to get a true assessment of its performance in real-world scenarios. The test dataset should ideally be kept entirely separate from the data used for training and validation.

3.The number of iterations, also referred to as the number of folds (k), can have an impact on the estimate of model performance obtained through cross-validation. Whether you get a better estimate with higher iterations depends on various factors, including the size of your dataset and the trade-offs between computational cost and estimation accuracy. Here are some considerations:

Smaller Datasets: For smaller datasets, having a higher number of iterations (folds) can be beneficial. This is because a smaller dataset provides fewer data points for training and validation, and increasing the number of iterations helps in using the available data more effectively. Common choices for small datasets are 5-fold or 10-fold cross-validation.

Larger Datasets: With larger datasets, the impact of increasing the number of iterations becomes less significant. In such cases, you can use a lower number of iterations to reduce computational costs without sacrificing the reliability of your estimate. 3-fold cross-validation is often used for larger datasets.

Computational Cost: Keep in mind that as the number of iterations increases, the computational cost of cross-validation also increases. Training and evaluating the model multiple times can be time-consuming and resource-intensive. Therefore, you need to strike a balance between the accuracy of the estimate and the available computational resources.

Stability of Results: A higher number of iterations can lead to more stable estimates, especially if your dataset is subject to randomness or if the validation performance varies significantly across different data splits. However, there are diminishing returns, and after a certain point, increasing the number of iterations might not significantly improve the estimate's stability.

Bias-Variance Trade-off: Cross-validation helps in estimating both bias and variance. With a higher number of iterations, you may get a slightly better estimate of the variance in your model's performance across different data splits. However, if your model has a significant bias problem, increasing the number of iterations won't address that issue; you'll need to improve your model's architecture or data preprocessing.

In summary, the effect of the number of iterations on the estimate of model performance through cross-validation depends on the dataset size, computational resources, and the need for stability in the estimates. It's essential to strike a balance that provides a reliable estimate without overburdening your computational infrastructure. It's also important to consider other factors like data quality, model complexity, and feature engineering in conjunction with the choice of cross-validation iterations to obtain a comprehensive assessment of your machine learning model's performance.

4.Increasing the number of iterations (folds) in cross-validation can help mitigate the challenges posed by very small training or validation datasets to some extent. However, there are limits to how much this can compensate for small dataset sizes, and it's essential to strike a balance and consider other factors:

Advantages of Increasing Iterations with Small Datasets:

Better Utilization: When you have a very small dataset, using more iterations allows you to make better use of the limited data you have. Each fold serves as both a validation and training set, ensuring that each data point contributes to the assessment of model performance.

Reduced Variance: With more iterations, you get a better estimate of the variance in your model's performance across different data splits. This can help you identify how stable your model is when trained on limited data.

Limitations and Considerations:

Computational Cost: While increasing iterations can be beneficial for small datasets, it also increases the computational cost. Training and evaluating the model multiple times can be time-consuming, which may not be practical in situations with severe computational constraints.

Limited Data: Cross-validation cannot magically create more data; it can only help you use the available data more effectively. If your dataset is extremely small, even with more iterations, you might still face challenges related to model overfitting, limited generalization, and difficulties in capturing complex patterns.

Data Quality: If your dataset is small, the quality of the data becomes even more critical. No amount of cross-validation iterations can compensate for issues like data imbalance, outliers, or noise. Preprocessing and cleaning the data remain essential.

Model Complexity: When dealing with very small datasets, it's often wise to choose simpler models or use regularization techniques to prevent overfitting, as complex models may struggle to generalize from such limited data.

In summary, increasing the number of iterations in cross-validation can help you get a more reliable estimate of model performance when dealing with very small training or validation datasets. However, it's not a panacea, and there are limits to what it can achieve. Additionally, it's crucial to consider computational constraints, data quality, and model complexity alongside the choice of cross-validation iterations to make the most informed decisions for your specific machine learning problem.